<font color="#a9a56c" size=2> **@Author: Muhammad Hammad** </font>

# Introduction to Asyncio in Python

**Real-World Hook:** Imagine you’re writing a web scraper that needs to fetch 100 pages. If each page takes 1 second, doing it one by one (synchronously) takes almost 2 minutes! Asyncio can overlap waits and finish much faster.

**Prerequisites:** Basic Python (functions, loops) and understanding of synchronous code.

## 1. What Happens Right Now?

When you write ordinary Python, it does one thing after another in order—this is called **synchronous** code.

* 🔄 You call a function, Python does it, you get the result, then move on.
* 🛑 If one step takes a long time (like waiting for a web page or reading a big file), everything else just sits and waits.

Think of it like making breakfast by yourself:

1. You fry the eggs,
2. Then you toast the bread,
3. Then you pour the coffee.
   You can’t start toasting until the eggs are done, and you can’t pour coffee until the toast is done.

---

## 2. What Could Be Better?

Imagine instead you have help or you multitask:

* You start frying the eggs,
* While the eggs are cooking you pop the bread in the toaster,
* While the toast is popping you pour the coffee.

No one has to stand around doing nothing—this is closer to **asynchronous** programming.

---

## 3. Asyncio in Python: The Basics

Python provides the **asyncio** library to let your code work on multiple things “at once,” even though there’s just one main thread. It does this by:

1. **Coroutines** – special functions you mark with `async def` that can pause themselves when they’re waiting.
2. **`await`** – tells Python “pause here and let other work happen until this I/O or timer is done.”
3. **Event Loop** – the background manager that jumps between paused coroutines, resuming them when they’re ready.
4. **Tasks** – wrappers around coroutines that schedule them to run.
5. **Futures** – placeholders for results that are not ready yet.




## Making `asyncio.run()` Work in Jupyter
Jupyter has its own event loop, so `asyncio.run()` alone fails:
```
RuntimeError: This event loop is already running
```

**Fix:**

In [ ]:
import nest_asyncio
nest_asyncio.apply()
# Now asyncio.run() works

**Alternative:** Use top-level `await main()` in IPython 7+ instead of `asyncio.run()`.

## Coffee & Pastry Example (Sync vs Async)
### Synchronous Version

In [ ]:
import time
def make_coffee_sync():
    time.sleep(2)
    print('Coffee ready')
    
def make_pastry_sync():
    time.sleep(3)
    print('Pastry ready')
    
start = time.time()
make_coffee_sync()
make_pastry_sync()
print(f'Sync total: {time.time()-start:.2f} seconds')

Coffee ready
Pastry ready
Sync total: 5.00 seconds


### Asynchronous Version

In [3]:
import asyncio
async def make_coffee_async():
    await asyncio.sleep(2)
    print('Coffee ready')
    
async def make_pastry_async():
    await asyncio.sleep(3)
    print('Pastry ready')
    
async def main():
    await asyncio.gather(make_coffee_async(), make_pastry_async())
    
# Run the async main function
start = asyncio.get_event_loop().time()
await main()
print(f'Async total: {asyncio.get_event_loop().time()-start:.2f} seconds')

Coffee ready
Pastry ready
Async total: 3.00 seconds


| Approach     | Total Time |
|--------------|------------|
| Synchronous  | ~5.00 s    |
| Asynchronous | ~3.00 s    |

**Quiz:** Why did async save ~2 seconds? (Hint: Overlapping waits)

## Best Practices & Common Functions ✅
- ✅ Always `await` coroutines
- ✅ Use `asyncio.run()` in scripts or `await` in notebooks
- ✅ Use `asyncio.gather()` to run many coroutines together
- ✅ Use `asyncio.create_task()` to fire-and-forget and check back later
- **Helpers:**
  - `asyncio.sleep(sec)`
  - `asyncio.create_task(coro)`
  - `asyncio.gather(*coros)`

## Understanding `asyncio` vs `await`
**Broken snippet** (fix it!):

In [ ]:
import asyncio
async def f():
    asyncio.sleep(1)
    print('Done')
asyncio.run(f())

## Checking for Awaitable Objects

In [22]:
import inspect
print(inspect.isawaitable(asyncio.sleep(1)))  # True
print(inspect.isawaitable(time.sleep(1)))      # False

C:\Users\PMLS\AppData\Local\Temp\ipykernel_25236\4077284270.py:2: RuntimeWarning: coroutine 'sleep' was never awaited
  print(inspect.isawaitable(asyncio.sleep(1)))  # True


True
False


## Custom Awaitables (`__await__`)

In [ ]:
class Countdown:
    def __init__(self, n): self.n = n
    def __await__(self):
        while self.n > 0:
            yield from asyncio.sleep(1).__await__()
            print(self.n)
            self.n -= 1
        return 'Lift off!'

async def main():
    result = await Countdown(3)
    print(result)
    
asyncio.run(main())

3
2
1
Lift off!


## Running Multiple Tasks (Interleaving)

In [25]:
import time, asyncio

async def t1():
    print(time.time(), 'Task1 start')
    await asyncio.sleep(2)
    print(time.time(), 'Task1 done')

async def t2():
    print(time.time(), 'Task2 start')
    await asyncio.sleep(1)
    print(time.time(), 'Task2 done')

async def main():
    await asyncio.gather(t1(), t2())

asyncio.run(main())

1754315793.1556597 Task1 start
1754315793.1556597 Task2 start
1754315794.1607387 Task2 done
1754315795.1567729 Task1 done


## Why Use Async? Key Benefits
- Efficient I/O (overlap waits)
- Single-threaded concurrency
- Keeps GUIs and servers responsive
- **Not for CPU-bound tasks** (use multiprocessing there)

## Futures Deep Dive
**States Diagram:**
```
Pending --> Done
       \--> Cancelled
```


In [27]:
async def example_future():
    fut = asyncio.Future()
    print('Pending:', fut.done())
    await asyncio.sleep(1)
    fut.set_result('Yep')
    print('Done:', fut.done())
    print('Result:', fut.result())
    
asyncio.run(example_future())

Pending: False
Done: True
Result: Yep


## What Is the Event Loop?
- The **engine** driving async code
- Manages task queue, I/O, timers, switching

**Additional methods:** `loop.run_until_complete()`, `loop.run_forever()`

In [28]:
loop = asyncio.get_event_loop()
print('Running:', loop.is_running())
task = loop.create_task(asyncio.sleep(1))
print(task)

Running: True
<Task pending name='Task-21' coro=<sleep() running at c:\Users\PMLS\anaconda3\envs\aiclass\Lib\asyncio\tasks.py:643>>
